<a href="https://colab.research.google.com/github/wisecrepin4/Terrace-detection/blob/main/Radical_terraces_Extraction_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Environment Setup & Geospatial Data Pipeline

I initialized this environment by mounting Google Drive for persistent access to my 600-image dataset and the heavy LiDAR rasters. I imported ResNet50 to serve as the feature extractor for my model, as its deep residual layers are excellent at identifying the subtle textures of terraces. I’ve also standardized my global parameters—setting a 256x256 image size and a 50-epoch training limit—to ensure the model has enough time to converge on the complex terrain features of the Rwandan landscape.


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import glob
import numpy as np
import tensorflow as tf
from google.colab import drive
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, UpSampling2D, Concatenate, Conv2D, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50


# Paths for your folders
LIDAR_PATH = '/content/drive/MyDrive/Lidar data/'
BASE_PATH = '/content/drive/MyDrive/Rwandan Radical Terraces/'
IMAGE_PATH = os.path.join(BASE_PATH, 'images/')
MASK_PATH  = os.path.join(BASE_PATH, 'masks/')
OUTPUT_PATH = os.path.join(BASE_PATH, 'predictions/')
os.makedirs(OUTPUT_PATH,exist_ok=True)
print(f"Predictions folder Is ready at: {OUTPUT_PATH}")


# Parameters
IMG_SIZE = (256, 256)
BATCH_SIZE = 8
EPOCHS = 50

Predictions folder Is ready at: /content/drive/MyDrive/Rwandan Radical Terraces/predictions/


## Data Preprocessing & Input Pipeline Optimization

I built this custom loading pipeline to solve the specific visual challenges of LiDAR data. I implemented per_image_standardization to normalize the 'grayish' contrast of the rasters, ensuring the model focuses on the shapes of the terraces rather than lighting variations. To keep the training process fast, I used the tf.data API with prefetching, which allows the CPU to prepare the next batch of images while the GPU is still processing the current one, significantly reducing bottlenecking

In [ ]:
def load_data(image_path, mask_path):
    # 1. Load Image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, IMG_SIZE)

    # 2. FORCE CONTRAST (Fixes the "Non-colorful/Flat" image problem)
    # We convert to grayscale and back to ensure the "Texture" is what the AI learns, not the color.
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.grayscale_to_rgb(image)
    image = tf.image.per_image_standardization(image)

    # 3. Load Mask
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, IMG_SIZE)
    # Ensure mask is strictly 0 or 1
    mask = tf.cast(mask > 127, tf.float32)

    return image, mask

# Dataset Preparation (Stays the same)
image_files = sorted(glob.glob(os.path.join(IMAGE_PATH, '*.png')))
mask_files = sorted(glob.glob(os.path.join(MASK_PATH, '*.png')))

train_images, test_images, train_masks, test_masks = train_test_split(
    image_files, mask_files, test_size=0.2, random_state=42)

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_masks))
train_dataset = train_dataset.shuffle(100).map(load_data).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

##ResUNet Architecture: Hybrid Encoder with Bilinear Decoder
I designed this architecture to merge the power of Transfer Learning with custom reconstruction logic. I leveraged a ResNet50 backbone as the encoder to extract deep features from the terrain. For the decoder, I made a critical engineering choice: I implemented Bilinear Interpolation in the UpSampling2D layers rather than standard transpose convolutions. This was specifically done to eliminate 'checkerboard artifacts' and ensure the thin, linear edges of the terraces remain smooth and continuous in the final prediction.


In [ ]:
def ResUNet(input_size=(256, 256, 3)):
    inputs = Input(input_size)
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)

    # Encoder (Your Logic)
    c1 = base_model.get_layer('conv1_relu').output
    c2 = base_model.get_layer('conv2_block3_out').output
    c3 = base_model.get_layer('conv3_block4_out').output
    c4 = base_model.get_layer('conv4_block6_out').output
    c5 = base_model.get_layer('conv5_block3_out').output

    # Decoder (concatenating again + Bilinear Fix)
    def up_block(x, skip, filters):
        # Using 'bilinear' interpolation removes the grid noise
        x = UpSampling2D((2,2), interpolation='bilinear')(x)
        x = Concatenate()([x, skip])
        x = Conv2D(filters, 3, activation='relu', padding='same')(x)
        x = BatchNormalization()(x) # Added for stability in grayscale
        x = Conv2D(filters, 3, activation='relu', padding='same')(x)
        return x

    u6 = up_block(c5, c4, 1024)
    u7 = up_block(u6, c3, 512)
    u8 = up_block(u7, c2, 256)
    u9 = up_block(u8, c1, 128)

    u10 = UpSampling2D((2,2), interpolation='bilinear')(u9)
    outputs = Conv2D(1, 1, activation='sigmoid')(u10)

    return Model(inputs, outputs)

model = ResUNet()
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='binary_crossentropy', # Or use Dice Loss for higher accuracy
              metrics=['accuracy'])

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


##  Automated Training Supervision & Convergence Logic

I configured the training process to run for up to 50 epochs, which provides the model ample time to learn the complex spatial patterns of Rwandan terraces. However, I didn't want the model to just run blindly; I implemented EarlyStopping with a 10-epoch patience. This means that if the AI reaches its peak accuracy and stops improving for 10 consecutive rounds, the system will automatically terminate the training and restore the absolute best version of the weights. This guarantees the highest possible precision for the LiDAR analysis without wasting computational resources.

In [ ]:

checkpoint_path = os.path.join(BASE_PATH, 'best_model.h5')

callbacks = [
    # Now it saves to Google Drive, not the temporary Colab folder
    tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True),
    tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
]

print("Starting training on Rwandan Radical Terraces...")
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    callbacks=callbacks
)

Starting training on Rwandan Radical Terraces...
Epoch 1/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6867 - loss: 0.6126

/usr/local/lib/python3.12/dist-packages/keras/src/callbacks/model_checkpoint.py:302: UserWarning: Can save best model only with val_loss available.
  if self._should_save_model(epoch, batch, logs, filepath):


60/60 ━━━━━━━━━━━━━━━━━━━━ 187s 1s/step - accuracy: 0.6876 - loss: 0.6112
Epoch 2/50


/usr/local/lib/python3.12/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 493ms/step - accuracy: 0.8187 - loss: 0.3949

...

60/60 ━━━━━━━━━━━━━━━━━━━━ 35s 578ms/step - accuracy: 0.9714 - loss: 0.0683
Epoch 47/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step - accuracy: 0.9739 - loss: 0.0617

60/60 ━━━━━━━━━━━━━━━━━━━━ 39s 649ms/step - accuracy: 0.9738 - loss: 0.0618
Epoch 48/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 501ms/step - accuracy: 0.9682 - loss: 0.0764

60/60 ━━━━━━━━━━━━━━━━━━━━ 60s 1s/step - accuracy: 0.9682 - loss: 0.0765  
Epoch 49/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 767ms/step - accuracy: 0.9692 - loss: 0.0759

60/60 ━━━━━━━━━━━━━━━━━━━━ 51s 847ms/step - accuracy: 0.9692 - loss: 0.0759
Epoch 50/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 502ms/step - accuracy: 0.9721 - loss: 0.0655

60/60 ━━━━━━━━━━━━━━━━━━━━ 39s 650ms/step - accuracy: 0.9721 - loss: 0.0655



##The Inference Engine: Processing Giant Images in Small Pieces


The LiDAR images of Rwanda are massive—too big for any computer to process all at once. To solve this, I built a Sliding Window Engine. It works like a magnifying glass: it scans the giant image piece-by-piece (256 pixels at a time), lets the AI predict the terraces for that small square, and then perfectly stitches them back together into one final map. I also added Reflective Padding to the edges so the AI doesn't get 'confused' when it hits the border of the file.



In [ ]:
def predict_full_lidar(path, model, patch_size=256):
    import cv2
    import numpy as np
    import tensorflow as tf

    full_img = cv2.imread(path)
    if full_img is None: return None
    h, w, _ = full_img.shape

    # 1. Padding
    pad_h = (patch_size - h % patch_size) % patch_size
    pad_w = (patch_size - w % patch_size) % patch_size
    img_padded = np.pad(full_img, ((0, pad_h), (0, pad_w), (0, 0)), mode='reflect')

    mask_pred = np.zeros((img_padded.shape[0], img_padded.shape[1]), dtype=np.float32)

    # 2. Inference Loop
    for i in range(0, img_padded.shape[0], patch_size):
        for j in range(0, img_padded.shape[1], patch_size):
            tile = img_padded[i:i+patch_size, j:j+patch_size]

            # Match Cell 2: Grayscale standardization
            tile_tf = tf.convert_to_tensor(tile)
            tile_tf = tf.image.rgb_to_grayscale(tile_tf)
            tile_tf = tf.image.grayscale_to_rgb(tile_tf)
            tile_standardized = tf.image.per_image_standardization(tile_tf).numpy()

            tile_input = np.expand_dims(tile_standardized, axis=0)
            pred = model.predict(tile_input, verbose=0)
            mask_pred[i:i+patch_size, j:j+patch_size] = pred[0, :, :, 0]

    # 3. Binary Threshold (Kill the Gray)
    raw_output = mask_pred[:h, :w]
    binary_mask = (raw_output > 0.5).astype(np.uint8) * 255

    # 4. AREA FILTER (Kill the Dots)
    # Connectivity 8 looks at all surrounding pixels
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(binary_mask, connectivity=8)

    clean_final = np.zeros(binary_mask.shape, dtype=np.uint8)

    # Only keep objects larger than 400 pixels (Real Terraces are usually huge)
    for i in range(1, nb_components):
        if stats[i, cv2.CC_STAT_AREA] >= 400:
            clean_final[output == i] = 255

    return clean_final

##Automated Batch Processing for Large-Scale Mapping

I designed this cell to automate the work. Since I have multiple large-scale LiDAR files, I created a Batch Pipeline that automatically finds every .tif file in my folder and processes them one by one. I used a Dictionary to keep the results organized in the computer's memory while simultaneously saving a permanent copy to the disk. This ensures that even if I'm working with hundreds of images, the system stays organized and no data is lost.

In [ ]:
import tensorflow as tf
import os

# 1. PATH TO YOUR SAVED BRAIN
checkpoint_path = os.path.join(BASE_PATH, 'best_model.h5')

# 2. THE RECOVERY LOGIC
if os.path.exists(checkpoint_path):
    print(f" Loading trained weights from: {checkpoint_path}")
    # This overwrites the blank model with your 'Best' version
    model.load_weights(checkpoint_path)
    print(" Model is now ready for high-accuracy prediction!")
else:
    print(f" ERROR: No saved model found at {checkpoint_path}")
    print(" ACTION: You must run the training cell first to create this file.")

🧠 Loading trained weights from: /content/drive/MyDrive/Rwandan Radical Terraces/best_model.h5
✅ Model is now ready for high-accuracy prediction!


In [ ]:
import cv2
import numpy as np
import os
import glob

# 1. THE REFINEMENT ENGINE (Fixes the jagged/missing spots)
def perfectionist_refine(mask):
    """
    Standardizes the AI mask to ensure professional GIS-ready results.
    """
    if mask is None: return None

    # Ensure binary format (Black and White)
    _, binary = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    # BRIDGE GAPS: Connects broken terrace segments
    kernel = np.ones((5,5), np.uint8)
    closed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

    # REMOVE NOISE: Filters out small artifacts/dots smaller than 200 pixels
    nb_components, output, stats, _ = cv2.connectedComponentsWithStats(closed, connectivity=8)
    final_mask = np.zeros(output.shape, dtype=np.uint8)
    for i in range(1, nb_components):
        if stats[i, cv2.CC_STAT_AREA] >= 200:
            final_mask[output == i] = 255
    return final_mask

# 2. THE PRODUCTION LOOP
# Use case-insensitive search to ensure orthophotos are 'seen'
os.makedirs(OUTPUT_PATH, exist_ok=True)
lidar_files = sorted(glob.glob(os.path.join(LIDAR_PATH, "*.tif")) +
                     glob.glob(os.path.join(LIDAR_PATH, "*.TIF")))

if not lidar_files:
    print(f" ERROR: No files found in {LIDAR_PATH}. Check your folder names/path.")
else:
    print(f" Processing {len(lidar_files)} images with ResUNet50 Accuracy...")
    all_results = {}

    for file_path in lidar_files:
        file_name = os.path.basename(file_path)
        print(f"--- Analyzing: {file_name} ---")

        # Core Prediction
        raw_mask = predict_full_lidar(file_path, model)

        if raw_mask is not None:
            # Accuracy Upgrade
            final_mask = perfectionist_refine(raw_mask)
            all_results[file_name] = final_mask

            # Save as Lossless PNG for Vectorization
            save_name = file_name.replace('.tif', '.png').replace('.TIF', '.png')
            save_path = os.path.join(OUTPUT_PATH, f"mask_{save_name}")
            cv2.imwrite(save_path, final_mask)
            print(f"Success: {save_name}")
        else:
            print(f"Failed to generate mask for {file_name}")

    print(" Batch Processing Complete. You can now run the Shapefile cell.")


🚀 Processing 3 images with ResUNet50 Accuracy...
--- Analyzing: Orthophoto1.tif ---
✅ Success: Orthophoto1.png
--- Analyzing: Orthophoto2.tif ---
✅ Success: Orthophoto2.png
--- Analyzing: Orthophoto3.tif ---
✅ Success: Orthophoto3.png
🏆 Batch Processing Complete. You can now run the Shapefile cell.


# Vectorisation for GIS visualisation
This cell executes the final transformation of AI-predicted masks into georeferenced ESRI Shapefiles. By inheriting spatial metadata from the source LiDAR and applying geometric simplification, we convert raw pixels into professional-grade, measurable GIS polygons with 100% spatial integrity

In [ ]:
import os
import glob
import cv2
import numpy as np
import geopandas as gpd
import rasterio
from rasterio import features
from shapely.geometry import shape

# 1. THE DEFINITION (Ensures the function exists in memory)
def mask_to_perfect_shapefile(mask_path, original_lidar_path, output_name):
    with rasterio.open(original_lidar_path) as src:
        transform = src.transform
        lidar_crs = src.crs

    mask = cv2.imread(mask_path, 0)
    if mask is None: return None
    mask = (mask > 127).astype(np.uint8)

    shapes = features.shapes(mask, mask=mask, transform=transform)

    records = []
    for geom, value in shapes:
        poly = shape(geom)
        records.append({
            'geometry': poly,
            'properties': {
                'ID': len(records) + 1,
                'Area_m2': round(poly.area, 2),
                'Class': 'Radical Terrace'
            }
        })

    if not records:
        return None

    gdf = gpd.GeoDataFrame(records, crs=lidar_crs)
    gdf['geometry'] = gdf.geometry.simplify(0.1)

    output_path = os.path.join(OUTPUT_PATH, f"{output_name}.shp")
    gdf.to_file(output_path, driver="ESRI Shapefile")
    return output_path

# 2. THE EXECUTION
lidar_files = sorted(glob.glob(os.path.join(LIDAR_PATH, "*.tif")))
mask_files = sorted(glob.glob(os.path.join(OUTPUT_PATH, "mask_*.png")))

print(f"Processing {len(mask_files)} masks into Shapefiles...")

for mask_file in mask_files:
    mask_filename = os.path.basename(mask_file)
    search_target = mask_filename.replace('mask_', '').replace('.png', '')

    match = [f for f in lidar_files if search_target in os.path.basename(f)]

    if match:
        final_shp = mask_to_perfect_shapefile(mask_file, match[0], f"VECTORS_{search_target}")
        if final_shp:
            print(f" Created: {os.path.basename(final_shp)}")
    else:
        print(f" No matching LiDAR found for {search_target}")

🚀 Processing 3 masks into Shapefiles...


✅ Created: VECTORS_Orthophoto1.shp
✅ Created: VECTORS_Orthophoto2.shp
✅ Created: VECTORS_Orthophoto3.shp
